In [1]:
# pip install c3d

In [2]:
import c3d, csv, os

In [3]:
counter = 0
filepaths = []
for root, dirs, files in os.walk('data'):
    if len(files) > 1:
        for file in files:
            if 'ST' not in file:
                counter += 1    
                filepath = root + "/" + file
                filepaths.append(filepath)
filepaths.sort()
print("Number of C3D files found:", counter)

Number of C3D files found: 1143


In [4]:
%%capture cap --no-stderr
# we only read the labels once from one file since they are the same in all files
file = "data/2014001/2014001_C1_01.c3d"
%run -i c3d-metadata $file

In [5]:
with open('output.txt', 'w') as f:
    f.write(cap.stdout)

with open('participants.csv', mode='w') as participants:
    writer = csv.writer(participants)
    header1 = "Participant Speed Trial "
    header2 = "# # # "
    header3 = " "
    r = open('output.txt')
    analog = 0
    for index, line in enumerate(r):
        if "ANALOG.LABELS[" in line:
            marker = line.split(' ')[2].split('\'')[1]
            for i in range(0, 15):
                header1 += marker + ' '
                header2 += str(i + 1) + ' '
            analog += 1
            if analog == 8:
                break
    writer.writerow(header1.split(' '))
    writer.writerow(header2.split(' '))
    writer.writerow(header3)

    # Frame data starts here
    #
    # There are 20 (300/15) analog variables (8 for EMG and 12 for Forceplate):
    #     analog_count: 300
    #     analog_per_frame: 15
    #
    count = 0
    for path in filepaths:
        handle = open(path, 'rb')
        reader = c3d.Reader(handle)
        ID = handle.name.split('/')[2].split('_')[0]
        speed = handle.name.split('_')[1][1]
        trial = handle.name.split('_')[2].split('.')[0][1]
        counter = 0
        row = ID + " " + speed + " " + trial + " "
        for item in reader.read_frames():
            # item[2] contains the array with all 300 analog variables
            counter += 1
            for i in range(0, 8): # item[2][0] to item[2][7] are the EMG analog values
                for j in range(0, 15):
                    row += str(item[2][i][j]) + ' '
            writer.writerow(row.split(' '))
            row = ID + " " + speed + " " + trial + " "
        count += 1
        # only write two participants to the CSV then exit ### comment out next 2 lines for full dataset
        if count == 45:
            break